<a href="https://colab.research.google.com/github/AccessDenied1/Cyclone_project/blob/master/Cyclone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from tqdm import tqdm

In [0]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)
#!/usr/bin/env python -W ignore::DeprecationWarning

In [0]:
%cd /content/drive/My\ Drive/Cyclone_project/rigid_body

/content/drive/My Drive/Cyclone_project/rigid_body


In [0]:
import cv2
from PIL import Image as Im
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow
image_dir = ['1.jpg' , '2.jpg' , '3.jpg' , '4.jpg']
data=[]
for i in image_dir:
    img = cv2.imread(i)
    img = cv2.resize(img , (400, 400))
    data.append(img)

In [0]:
import numpy as np

In [0]:
#plotting the rigid body motion ,four images

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from google.colab.patches import cv2_imshow
import matplotlib.animation as an
import numpy as np
import cv2
import os
import time
for i in data:
    cv2_imshow(i)

In [0]:
img_data=[]
img = data[0]
rows = img.shape[0]
cols = img.shape[1]
img_center = (cols / 2, rows / 2)
for i in range(0,90):
    M = cv2.getRotationMatrix2D(img_center, i, 1)
    rotated_image = cv2.warpAffine(img, M, (cols, rows), borderValue=(255,255,255))
    img_data.append(rotated_image)
    #cv2_imshow(rotated_image)

In [0]:
images=[]
for i in img_data:
    images.append(np.array(i))

# Gunnar Farneback method  


In [0]:
frame1 = images[0].copy()
prvs= cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)

In [0]:
hsv = np.zeros_like(frame1)

In [0]:
hsv[...,1] = 255
#print(hsv)

In [0]:
i=2
while(i<20):
    frame2 = images[i].copy()
    next = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
    #Getting the flow information by comparing the two consecutive images(1st and second) given above
    flow = cv2.calcOpticalFlowFarneback(prvs,next,None,0.5,3,43,3,7,1.5,0) 
    #Gettint the magnitude and angle of the velocity vector
    mag,ang = cv2.cartToPolar(flow[...,0],flow[...,1])
    hsv[...,0] = ang*180 / np.pi/2
    hsv[...,2] = cv2.normalize(mag,None,0,255,cv2.NORM_MINMAX)
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break
    prvs = next
    i=i+2

In [0]:
#Getting velocity in x and y direction(list x_vel and y_vel)th 
x_vel = np.zeros((400,400) , dtype = float)
y_vel = np.zeros((400,400) , dtype = float)
for i in range(400):
    for j in range(400):
        a,b = flow[i][j]
        x_vel[i][j] = a
        y_vel[i][j] = b

In [0]:
print(np.max(y_vel))
print(np.max(x_vel))

0.7875387072563171
0.6441136598587036


In [0]:
# x_pos = np.zeros((100,100))
# y_pos = np.zeros((100,100))
# for i in range(100):
#     for j in range(100):
#         x_pos[i][j] = i
#         y_pos[i][j] = j

In [0]:
# #plotting 
# fig, ax = plt.subplots(figsize=(20,20))
# ax.axis([0,100,0,100])
# #ax.tick_params(length = 1)
# #ax.xaxis.set_ticks([1.,2.,3.,100.])
# ax.quiver(x_pos,y_pos,x_vel,y_vel, scale=10)
# plt.show()

In [0]:
# method-2
import numpy as np
from scipy import signal
def optical_flow(I1g, I2g, window_size, tau=1e-2):
 
    kernel_x = np.array([[-1., 1.], [-1., 1.]])
    kernel_y = np.array([[-1., -1.], [1., 1.]])
    kernel_t = np.array([[1., 1.], [1., 1.]])#*.25
    w = int(window_size/2) # window_size is odd, all the pixels with offset in between [-w, w] are inside the window
    I1g = I1g / 255. # normalize pixels
    I2g = I2g / 255. # normalize pixels
    # Implement Lucas Kanade
    # for each point, calculate I_x, I_y, I_t
    mode = 'same'
    fx = signal.convolve2d(I1g, kernel_x, boundary='symm', mode=mode)
    fy = signal.convolve2d(I1g, kernel_y, boundary='symm', mode=mode)
    ft = signal.convolve2d(I2g, kernel_t, boundary='symm', mode=mode) + signal.convolve2d(I1g, -kernel_t, boundary='symm', mode=mode)
    u = np.zeros(I1g.shape)
    v = np.zeros(I1g.shape)
    # within window window_size * window_size
    #print("Type of w = ",type(w) , type(I1g.shape[0]-w))
    for i in range(w, I1g.shape[0]-w):
        for j in range(w, I1g.shape[1]-w):
            Ix = fx[i-w:i+w+1, j-w:j+w+1].flatten()
            Iy = fy[i-w:i+w+1, j-w:j+w+1].flatten()
            It = ft[i-w:i+w+1, j-w:j+w+1].flatten()
            b = np.reshape(It, (It.shape[0],1)) # get b here
            A = np.vstack((Ix, Iy)).T # get A here

            if np.min(abs(np.linalg.eigvals(np.matmul(A.T, A)))) >= tau:
                nu = np.matmul(np.linalg.pinv(A), b) # get velocity here
                u[i,j]=nu[0]
                v[i,j]=nu[1]
    return (u,v)

In [0]:
# fr1 = images[0].copy()
# prvs= cv2.cvtColor(fr1, cv2.COLOR_BGR2GRAY)
# fr2 = images[2].copy()
# net = cv2.cvtColor(fr2, cv2.COLOR_BGR2GRAY)
# (u,v) = optical_flow(prvs, net, 16, tau=1e-2)

In [0]:
# np.max(u)

In [0]:
# #plotting 
# fig, ax = plt.subplots(figsize=(20,20))
# ax.axis([0,100,0,100])
# plt.quiver(x_pos[:],y_pos[:],u[:],v[:],color="red",angles="xy",scale_units="xy",scale=1,)
# plt.show()

In [0]:
# def encode(i,j):
#     return i+(j*400)
# def decode(num):
#     i = num%400
#     j = (num - i)/400
#     return i,j

In [0]:
# Ref_img = np.zeros((400,400,2) , dtype=np.float32)
# labels = np.zeros((5000,2))
# #labels = []
# Train_x = []
# k=0
# #Dataset Generation from Gunnar Farneback method
# for i in tqdm(range(400)):
#     for j in range(400):
#         Ref_y = y_vel[(399-i) : (799-i) , (399-j) : (799-j)]
#         Ref_x = x_vel[(399-i) : (799-i) , (399-j) : (799-j)]
#         Ref_img[:,:,0] = Ref_x
#         Ref_img[:,:,1] = Ref_y
#         #Ref_img[:,:,2] = np.zeros((400,400))
#         labels[k][0] = i/400.0
#         labels[k][1] = j/400.0
#         Train_x.append(Ref_img.copy())
#         k=k+1
#         if(k==5000):
#             break
#     if(k==5000):
#         break

In [0]:
# from sklearn.model_selection import train_test_split
# X_train, X_Valid, y_train, y_valid = train_test_split(Train_x, labels, test_size=0.10, random_state=42)
# X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.10, random_state=42)

In [0]:
# print("X_Valid = ",len(X_Valid),", Y_valid = ",y_valid.shape)
# print("X_Train = ",len(X_train),", Y_train = ",y_train.shape)

In [0]:
from keras.layers import Input,Conv2D,MaxPooling2D,Flatten,Dense
from keras.models import Model

Using TensorFlow backend.


In [0]:
inputs = Input(shape=(200,200,2))
x = Conv2D(32,(3,3) ,padding = 'same' , activation='relu')(inputs)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Conv2D(64, (3, 3) , activation='relu')(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Conv2D(128, (3, 3) , activation='relu')(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Flatten()(x)
x = Dense(128 , activation='relu')(x)
out1 = Dense(2 , activation='linear')(x)
model = Model(inputs = inputs, outputs = out1)

In [0]:
#from keras.applications.resnet import ResNet50
#from keras.layers import GlobalAveragePooling2D , Dropout

In [0]:
#base_model = ResNet50(include_top=False, weights='imagenet', input_tensor=None, input_shape=(400,400,3))

In [0]:
# x = base_model.output
# x = GlobalAveragePooling2D(name='avg_pool')(x)
# x = Dense(1024, activation='relu')(x)
# x= Dropout(0.2)(x)
# x = Dense(256, activation='relu')(x)
# x= Dropout(0.2)(x)
# x = Dense(64, activation='relu')(x)
# x= Dropout(0.2)(x)
# x = Dense(16, activation='relu')(x)
# x= Dropout(0.2)(x)
# predictions = Dense(2, activation='softmax')(x)
# model = Model(inputs=base_model.input, outputs=predictions)


In [0]:
# import keras
# adam = keras.optimizers.Adam(lr=0.01, beta_1=0.9, beta_2=0.999, amsgrad=False)

In [0]:
model.compile(loss='mse',
              optimizer='adam',
              metrics=['mse','mae','accuracy'])
print(model.summary())


Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 200, 200, 2)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 200, 200, 32)      608       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 100, 100, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 98, 98, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 49, 49, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 47, 47, 128)       73856     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 23, 23, 128)       0  

#Sanity Checking of model

In [0]:
# X_Sanity_data = np.array(X_train[0:1000])
# Y_Sanity_data = np.array(y_train[0:1000])

In [0]:
# Sanity_data.shape

In [0]:
# history = model.fit(x=np.array(X_train), y=y_train , verbose = 1 , epochs = 20 , shuffle = True,validation_data=(np.array(X_Valid),y_valid))

In [0]:
# h= model.evaluate(x=np.array(X_test),y=y_test)

In [0]:
import time

In [0]:
import tensorflow as tf
import gc

In [0]:
train_size = 140000
valid_size = 16000
test_size = 4000

Train : 0 ~ 140000<br/>
Valid : 140000 ~ 156000<br/>
Test : 156000 ~ 160000<br/>

In [0]:
index_list = []
for i in range(400):
    for j in range(400):
        index_list.append([i,j])

In [0]:
Ref_img = np.zeros((200,200,2) , dtype=np.float32)
labels = np.zeros((40000,2))
Train_x = []
k=0
#Dataset Generation from Gunnar Farneback method
for i in range(200):
    for j in range(200):
        Ref_y = y_vel[199-i : 399-i , 199-j : 399-j]
        Ref_x = x_vel[199-i : 399-i , 199-j : 399-j]
        Ref_img[:,:,0] = Ref_x
        Ref_img[:,:,1] = Ref_y
        #labels.append([i/400.0 , j/400.0])
        #labels[k] = (encode(i,j)/159999.0)
        labels[k][0] = i/200.0
        labels[k][1] = j/200.0
        Train_x.append(Ref_img.copy())
        k=k+1

In [0]:
# def get_Chunk_Data(st,size):
#     Ref_img = np.zeros((200,200,2) , dtype=np.float32)
#     labels = np.zeros((size,2))
#     Train_x = []
#     k=0
#     for i in range(st,size+st):
#         a = index_list[i][0]
#         b = index_list[i][1]
#         Ref_y = y_vel[(199-a) : (399-a) , (199-b) : (399-b)]
#         Ref_x = x_vel[(199-a) : (399-a) , (199-b) : (399-b)]
#         Ref_img[:,:,0] = Ref_x
#         Ref_img[:,:,1] = Ref_y
#         labels[k][0] = i/200.0
#         labels[k][1] = j/200.0
#         Train_x.append(Ref_img.copy())
#         k=k+1
#     return Train_x,labels

In [0]:
# #x_data = np.zeros((3200, 400, 400, 3), dtype = np.float32)
# for ep in range(10):
#     count = 0
#     acc = 1
#     loss = 1
#     mean_absolute_error = 1
#     mean_squared_error = 1
#     val_acc = 1
#     val_loss = 1
#     val_mean_absolute_error = 1
#     val_mean_squared_error = 1
#     st_train = 0
#     st_vd = 140000
#     chunk_size = 4000
#     train_size = 140000
#     valid_size = 16000
#     valid_chunk = 2000
#     while(st_train<train_size):
#         gc.collect()
#         data,label = get_Chunk_Data(st_train,chunk_size)
#         val_data ,labl  = get_Chunk_Data(st_vd , valid_chunk)
#         x_data = np.array(data)
#         y_label = label.copy()
#         valid_data = np.array(val_data)
#         valid_label = labl.copy()
#         history = model.fit(x = x_data , y = y_label , verbose = 0 , epochs = 1 , shuffle = True , batch_size = 32 , validation_data = (valid_data,valid_label))
#         time.sleep(5)
#         gc.collect()
#         acc = min(acc , history.history['acc'][0])
#         loss = min(loss , history.history['loss'][0])
#         mean_absolute_error = min(mean_absolute_error , history.history['mean_absolute_error'][0])
#         mean_squared_error = min(mean_squared_error , history.history['mean_squared_error'][0])
#         val_acc = min(val_acc , history.history['val_acc'][0])
#         val_loss = min(val_loss , history.history['val_loss'][0])
#         val_mean_absolute_error = min(val_mean_absolute_error , history.history['val_mean_absolute_error'][0])
#         val_mean_squared_error = min(val_mean_squared_error , history.history['val_mean_squared_error'][0])
#         count = count + 1
#         print("count = ",count)
#         st_train = st_train+((int)(chunk_size/2))
#         st_vd = st_vd + ((int)(valid_chunk/2))
#         if(st_vd >= 156000):
#             st_vd = 140000
#         gc.collect()
#     print ("-----Epoch {}---------".format(ep+1))
#     print("------Minimum acc is {}------".format(acc))
#     print("------Minimum loss is {}------".format(loss))
#     print("------Minimum mean_absolute_error is {}------".format(mean_absolute_error))
#     print("------Minimum mean_squared_error is {}------".format(mean_squared_error))
#     print("------Minimum val_acc is {}------".format(val_acc))
#     print("------Minimum val_loss is {}------".format(val_loss))
#     print("------Minimum val_mean_absolute_error is {}------".format(val_mean_absolute_error))
#     print("------Minimum val_mean_squared_error is {}------".format(val_mean_squared_error))

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_Valid, y_train, y_valid = train_test_split(Train_x, labels, test_size=0.10, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.10, random_state=42)

In [0]:
print("X_Valid = ",len(X_Valid),", Y_valid = ",y_valid.shape)
print("X_Train = ",len(X_train),", Y_train = ",y_train.shape)

X_Valid =  4000 , Y_valid =  (4000, 2)
X_Train =  32400 , Y_train =  (32400, 2)


In [0]:
X_train = np.array(X_train[:])
y_train = np.array(y_train[:])
X_Valid = np.array(X_Valid[:])
y_valid = np.array(y_valid[:])

In [0]:
import gc
import time
time.sleep(5)
gc.collect()

168

In [0]:
del Train_x
del labels

In [0]:
model.fit(x=X_train , y = y_train , verbose=1,epochs=10,shuffle=True , batch_size = 32,validation_data=(X_Valid,y_valid))




Train on 32400 samples, validate on 4000 samples
Epoch 1/10





32400/32400 [==============================] - 42s 1ms/step - loss: 0.0024 - mean_squared_error: 0.0024 - mean_absolute_error: 0.0177 - acc: 0.9764 - val_loss: 6.2847e-05 - val_mean_squared_error: 6.2847e-05 - val_mean_absolute_error: 0.0066 - val_acc: 0.9848
Epoch 2/10
32400/32400 [==============================] - 32s 991us/step - loss: 3.2036e-04 - mean_squared_error: 3.2036e-04 - mean_absolute_error: 0.0108 - acc: 0.9862 - val_loss: 9.6717e-05 - val_mean_squared_error: 9.6717e-05 - val_mean_absolute_error: 0.0070 - val_acc: 0.9910
Epoch 3/10
32400/32400 [==============================] - 32s 983us/step - loss: 5.3613e-05 - mean_squared_error: 5.3613e-05 - mean_absolute_error: 0.0052 - acc: 0.9946 - val_loss: 3.1027e-05 - val_mean_squared_error: 3.1027e-05 - val_mean_absolute_error: 0.0043 - val_acc: 0.9930
Epoch 4/10
32400/32400 [==============================] - 32s 989us/step - loss: 1.6399e-04 - mean_squared_err

In [0]:
model.save('200x200_10_epoch.model')

In [0]:
model.evaluate(x = np.array(X_test),y=y_test)

3600/3600 [==============================] - 1s 366us/step


[7.96196563816112e-05,
 7.96196563816112e-05,
 0.0069363723446925485,
 0.9958333333333333]